In [ ]:
import psycopg2 as ps
import ipyleaflet as ipyl
import ipywidgets as ipyw
import pandas as pd
import json


In [ ]:
conn_string = "dbname='gis' user='postgres' password='123456' host='localhost'"
conn = ps.connect(conn_string)
cur = conn.cursor()
query = """SELECT 
               * 
           FROM 
               planet_osm_line 

"""
columns = """
        operator,
        name,
        highway,
        amenity,
        public_transport,
        railway,
        power,
        bicycle,
        barrier
"""
query_json_line = """
SELECT row_to_json(fc)
 FROM ( SELECT 'FeatureCollection' As type, array_to_json(array_agg(f)) As features
 FROM (SELECT 'Feature' As type
    , ST_AsGeoJSON(ST_Transform(lg.way, 4326))::json As geometry
    , row_to_json((SELECT l FROM (SELECT {}) As l
      )) As properties
   FROM planet_osm_line As lg ) As f )  As fc;
""".format(columns)

query_json_point = """
SELECT row_to_json(fc)
 FROM ( SELECT 'FeatureCollection' As type, array_to_json(array_agg(f)) As features
 FROM (SELECT 'Feature' As type
    , ST_AsGeoJSON(ST_Transform(lg.way, 4326))::json As geometry
    , row_to_json((SELECT l FROM (SELECT {}) As l
      )) As properties
   FROM planet_osm_point As lg order by random() limit 100) As f )  As fc;
""".format(columns)


cur.execute(query_json_point)
d_points = cur.fetchone()
cur.execute(query_json_line)
d_lines = cur.fetchone()

#df = pd.read_sql(query, conn)
conn.close()

In [ ]:
map = ipyl.Map(center=[52.52, 13.45], zoom=10)
label = ipyw.Label(layout=ipyw.Layout(width='100%'))

layer_points = ipyl.GeoJSON(data=d_points[0], hover_style={'fill_color': 'red'}, style={'fill_color':'#001010','fill':True})
layer_lines = ipyl.GeoJSON(data=d_lines[0],style={'color':'red'})

def hover_handler(event=None, id=None, properties=None):
    s = ""
    for key, value in properties.items():
        s = s +str(key) +":" + str(value) + ";\t"
    label.value = s
    
layer_lines.on_hover(hover_handler)
layer_points.on_click(hover_handler)
map.add_layer(layer_lines)
map.add_layer(layer_points)

ipyw.VBox([map, label])

